In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data

/home/calin/anaconda3/envs/mlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
batch_size = 128
z_dim = 10

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [5]:
def discriminator(input):
    '''
    input = [28*28, batch_size] array
    return the prob of being real 
    '''
    h0 = tf.layers.dense(input, 28*28, activation = tf.nn.relu, name='d0')
    h1 = tf.layers.dense(h0, 1,  name='d1')
    #prob = tf.sigmoid(h1, 'd2')
    
    return h1

def generator(inputs):
    '''
    receives random noise and tries to reproduce mnist image
    '''
    h0 = tf.layers.dense(inputs, 100, activation=tf.nn.relu, name='g0')
    h1 = tf.layers.dense(h0, 28*28, name='g1')
    h2 = tf.sigmoid(h1, name='g2')
    return h2


with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=[None, 100], name='z')
    fake_samples = generator(z)
   
    
with tf.variable_scope('D') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784], name='inputs')
    D_real = discriminator(x)
    scope.reuse_variables()
    D_fake = discriminator(fake_samples)
    

vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

loss_d =  tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
loss_g = - tf.reduce_mean(D_fake)

    
d_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(-loss_d, var_list=d_params))
g_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(loss_g, var_list=g_params))
    

clip_d = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in d_params]


In [6]:
num_steps = 100000
saver = tf.train.Saver()
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(num_steps):
        
        if i % 1000 == 0:
            save_path = saver.save(sess, "../tmp/model.ckpt")

        
        if i % 100 == 0:
            fake_data = sess.run(fake_samples, feed_dict={z: sample_Z(16, 100)})
            fig = plot(fake_data)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            plt.close(fig)

        for n_critic in range(5):
            train_data, _ = mnist.train.next_batch(batch_size)

            #update discriminator
            loss_disc, _, _ = sess.run([loss_d, d_solver, clip_d], {
                x: train_data,
                z: sample_Z(batch_size, 100)
            })

        #update generator
        loss_gen, _ = sess.run([loss_g, g_solver], {
            z: sample_Z(batch_size, 100)
        })
        
        
        if i % 100 == 0:
            print(loss_disc, loss_gen)
    

0.00434502 0.00745878
11.3483 8.13597
11.0363 7.43933
9.79727 6.1947
8.83342 5.21307
7.37464 4.78204
5.73522 4.13033
3.64517 3.35995
2.82704 3.33299
1.99089 2.96134
1.27473 2.31914
0.612264 0.648354
0.369925 0.472531
0.214903 0.245617
0.0936367 0.0168418
0.0751144 -0.276823
0.030008 0.144279
0.0110491 -0.0106224
0.0381925 -0.556144
0.00668149 0.233078
-0.00767229 0.146976
0.0233154 -0.386695
0.0182204 0.0511804
0.0229655 0.200304
-0.00799182 0.281355
-0.00462574 0.167975
-0.00624807 -0.253361
-0.000370696 -0.0433009
0.00869478 -0.0383293
0.0659612 0.691485
0.462027 0.692399
0.417209 0.26069
0.424618 0.111338
0.538002 -0.196953
0.477783 -0.279142
0.415308 -0.0350649
0.364697 -0.0999217
0.34108 -0.112726
0.32893 -0.0934143
0.350169 -0.205117
0.342521 -0.02816
0.385451 -0.165697
0.370145 -0.00723583
0.321212 -0.28148
0.302854 -0.0942789
0.299042 0.125821
0.356587 0.0827101
0.336692 -0.0140907
0.319865 0.162359
0.272896 0.0622155
0.259763 0.0161997
0.240216 0.0069924
0.217789 -0.037988
0.2

KeyboardInterrupt: 

In [ ]:
tf.reset_default_graph()

In [ ]:
#vanilla gan
"""
def discriminator(input):
    '''
    input = [28*28, batch_size] array
    return the prob of being real 
    '''
    h0 = tf.layers.dense(input, 28*28, activation = tf.nn.relu, name='d0')
    h1 = tf.layers.dense(h0, 1,  name='d1')
    prob = tf.sigmoid(h1, 'd2')
    
    return prob

def generator(inputs):
    '''
    receives random noise and tries to reproduce mnist image
    '''
    h0 = tf.layers.dense(inputs, 100, activation=tf.nn.relu, name='g0')
    h1 = tf.layers.dense(h0, 28*28, name='g1')
    h2 = tf.sigmoid(h1, name='g2')
    return h2


with tf.variable_scope('G'):
    z = tf.placeholder(tf.float32, shape=[None, 100], name='z')
    fake_samples = generator(z)
   
    
with tf.variable_scope('D') as scope:
    x = tf.placeholder(tf.float32, shape=[None, 784], name='inputs')
    D1 = discriminator(x)
    scope.reuse_variables()
    D2 = discriminator(fake_samples)
    

vars = tf.trainable_variables()
d_params = [v for v in vars if v.name.startswith('D/')]
g_params = [v for v in vars if v.name.startswith('G/')]

loss_d = - tf.reduce_mean(tf.log(D1) + tf.log(1.0 - D2))
loss_g = - tf.reduce_mean(tf.log(D2))
    
d_optimizer = tf.train.AdamOptimizer().minimize(loss_d, var_list=d_params)
g_optimizer = tf.train.AdamOptimizer().minimize(loss_g, var_list=g_params)
    
"""